Inicjalizacja środowiska Google Colab

*Wykonaj tylko raz, nie resetuje się wraz ze zmiennymi środowiskowymi*


In [ ]:
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL

from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:root@/")
engine.execute("CREATE DATABASE car_rental") #create db
engine.execute("USE car_rental") # select new db

 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.
     |████████████████████████████████| 51kB 1.5MB/s 


Pierwszy blok zadaniowy

Stwórz tabele *cars, clients, bookings* według wytycznych(bez relacji):

*   cars: car_id(int, pk), producer(str), model(str), year(int), horse_power(int), price_per_day(int)
*   clients: client_id(int, pk), name(str), surname(str), address(str), city(str)
*   bookings: booking_id(int, pk), client_id(int), car_id(int), start_date(date), end_date(date), total_amount(int)

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Integer, Date

eng = create_engine("mysql+pymysql://root:root@/car_rental", echo = True)

base = declarative_base()

class Cars(base):
   __tablename__ = 'cars'

   car_id = Column(Integer, primary_key=True, autoincrement=True)
   producer = Column(String(30), nullable=False)
   model = Column(String(30), nullable=False)
   year = Column(Integer, nullable=False)
   horse_power = Column(Integer, nullable=False)
   price_per_day = Column(Integer, nullable=False)

class Clients(base):
    __tablename__ = 'clients'

    client_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(30), nullable=False)
    surname = Column(String(30), nullable=False)
    address = Column(String(30), nullable=False)
    city = Column(String(30), nullable=False)

class Bookings(base):
    __tablename__ = 'bookings'

    booking_id = Column(Integer, primary_key=True, autoincrement=True)
    client_id = Column(Integer, nullable=False)
    car_id = Column(Integer, nullable=False)
    start_date = Column(Date, nullable=False)
    end_date = Column(Date, nullable=False)
    total_amount = Column(Integer, nullable=False)

base.metadata.create_all(eng)

2020-10-16 22:50:43,229 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2020-10-16 22:50:43,230 INFO sqlalchemy.engine.base.Engine {}
2020-10-16 22:50:43,236 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2020-10-16 22:50:43,237 INFO sqlalchemy.engine.base.Engine {}
2020-10-16 22:50:43,244 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2020-10-16 22:50:43,246 INFO sqlalchemy.engine.base.Engine {}
2020-10-16 22:50:43,249 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2020-10-16 22:50:43,250 INFO sqlalchemy.engine.base.Engine {}
2020-10-16 22:50:43,254 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2020-10-16 22:50:43,255 INFO sqlalchemy.engine.base.Engine {}
2020-10-16 22:50:43,258 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2020-10-16 22:50:43,259 INFO sqlalchemy.engine.base.E

To samo zadanie można wykonać za pomocą odpowiednich zapytań SQLa:

In [ ]:
sql_statement = 'CREATE TABLE sql_booking(booking_id INTEGER PRIMARY KEY,' \
'client_id INTEGER,car_id INTEGER,start_date DATE,end_date DATE,total_amount INTEGER);'

eng.execute(sql_statement)

# Check
result = eng.execute("SHOW TABLES;")
for description in result:
    print(description)

sql_statement = 'DROP TABLE sql_booking'
eng.execute(sql_statement)

2020-10-16 22:45:54,704 INFO sqlalchemy.engine.base.Engine CREATE TABLE sql_booking(booking_id INTEGER PRIMARY KEY,client_id INTEGER,car_id INTEGER,start_date DATE,end_date DATE,total_amount INTEGER);
2020-10-16 22:45:54,705 INFO sqlalchemy.engine.base.Engine {}
2020-10-16 22:45:54,742 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-16 22:45:54,744 INFO sqlalchemy.engine.base.Engine SHOW TABLES;
2020-10-16 22:45:54,745 INFO sqlalchemy.engine.base.Engine {}
('bookings',)
('cars',)
('clients',)
('sql_booking',)
2020-10-16 22:45:54,751 INFO sqlalchemy.engine.base.Engine DROP TABLE sql_booking
2020-10-16 22:45:54,752 INFO sqlalchemy.engine.base.Engine {}
2020-10-16 22:45:54,774 INFO sqlalchemy.engine.base.Engine COMMIT


Kolejne zadanie to dodanie kilku instancji do naszej tabeli

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=eng)
session = Session()
client_1 = Clients(name='Jan', surname='Kowalski', address='ul. Florianska 12', city='Krakow')
car_1 = Cars(producer='Seat', model='Leon', year=2016, horse_power=80, price_per_day=200)

session.add(client_1)
session.add(car_1)
session.commit()

2020-10-16 22:43:06,932 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-16 22:43:06,935 INFO sqlalchemy.engine.base.Engine INSERT INTO cars (producer, model, year, horse_power, price_per_day) VALUES (%(producer)s, %(model)s, %(year)s, %(horse_power)s, %(price_per_day)s)
2020-10-16 22:43:06,936 INFO sqlalchemy.engine.base.Engine {'producer': 'Seat', 'model': 'Leon', 'year': 2016, 'horse_power': 80, 'price_per_day': 200}
2020-10-16 22:43:06,944 INFO sqlalchemy.engine.base.Engine INSERT INTO clients (name, surname, address, city) VALUES (%(name)s, %(surname)s, %(address)s, %(city)s)
2020-10-16 22:43:06,955 INFO sqlalchemy.engine.base.Engine {'name': 'Jan', 'surname': 'Kowalski', 'address': 'ul. Florianska 12', 'city': 'Krakow'}
2020-10-16 22:43:06,959 INFO sqlalchemy.engine.base.Engine COMMIT


W kolejnym należy wyświetlić zawartość poszczególnych tabel

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=eng)
session = Session()

for client in session.query(Clients).all():
    print(client)

for car in session.query(Cars).all():
    print(car)


2020-10-16 22:51:43,926 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-16 22:51:43,928 INFO sqlalchemy.engine.base.Engine SELECT clients.client_id AS clients_client_id, clients.name AS clients_name, clients.surname AS clients_surname, clients.address AS clients_address, clients.city AS clients_city 
FROM clients
2020-10-16 22:51:43,930 INFO sqlalchemy.engine.base.Engine {}
2020-10-16 22:51:43,935 INFO sqlalchemy.engine.base.Engine SELECT cars.car_id AS cars_car_id, cars.producer AS cars_producer, cars.model AS cars_model, cars.year AS cars_year, cars.horse_power AS cars_horse_power, cars.price_per_day AS cars_price_per_day 
FROM cars
2020-10-16 22:51:43,936 INFO sqlalchemy.engine.base.Engine {}
<Car: id=1, producer=Seat,model=Leon, year=2016,horse_power=80, price_per_day=200>
<Car: id=2, producer=Seat,model=Leon, year=2016,horse_power=80, price_per_day=200>


W celu zwiększenia czytelności warto zdefiniować reprezentację stringową naszych obiektów.

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Integer, Date

eng = create_engine("mysql+pymysql://root:root@/car_rental", echo = True)

base = declarative_base()

class Cars(base):
   __tablename__ = 'cars'

   car_id = Column(Integer, primary_key=True, autoincrement=True)
   producer = Column(String(30), nullable=False)
   model = Column(String(30), nullable=False)
   year = Column(Integer, nullable=False)
   horse_power = Column(Integer, nullable=False)
   price_per_day = Column(Integer, nullable=False)

   def __repr__(self):
       return f'<Car: id={self.car_id}, producer={self.producer},' \
              f'model={self.model}, year={self.year},' \
              f'horse_power={self.horse_power}, price_per_day={self.price_per_day}>'


class Clients(base):
    __tablename__ = 'clients'

    client_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(30), nullable=False)
    surname = Column(String(30), nullable=False)
    address = Column(String(30), nullable=False)
    city = Column(String(30), nullable=False)

class Bookings(base):
    __tablename__ = 'bookings'

    booking_id = Column(Integer, primary_key=True, autoincrement=True)
    client_id = Column(Integer, nullable=False)
    car_id = Column(Integer, nullable=False)
    start_date = Column(Date, nullable=False)
    end_date = Column(Date, nullable=False)
    total_amount = Column(Integer, nullable=False)

base.metadata.create_all(eng)

2020-10-16 22:51:48,168 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2020-10-16 22:51:48,170 INFO sqlalchemy.engine.base.Engine {}
2020-10-16 22:51:48,176 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2020-10-16 22:51:48,178 INFO sqlalchemy.engine.base.Engine {}
2020-10-16 22:51:48,182 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2020-10-16 22:51:48,183 INFO sqlalchemy.engine.base.Engine {}
2020-10-16 22:51:48,185 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2020-10-16 22:51:48,188 INFO sqlalchemy.engine.base.Engine {}
2020-10-16 22:51:48,191 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2020-10-16 22:51:48,192 INFO sqlalchemy.engine.base.Engine {}
2020-10-16 22:51:48,195 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2020-10-16 22:51:48,196 INFO sqlalchemy.engine.base.E

Teraz uzyskamy troszkę bardziej czytelne wyniki:

Przy okazji w taki sposób można wykorzystać select

In [ ]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

Session = sessionmaker(bind=eng)
session = Session()

s = select([Clients]) # Make sql_statement
print(s)

for row in eng.execute(s):
	print(row)

for car in session.query(Cars).all():
    print(car)

SELECT clients.client_id, clients.name, clients.surname, clients.address, clients.city 
FROM clients
2020-10-16 22:57:26,251 INFO sqlalchemy.engine.base.Engine SELECT clients.client_id, clients.name, clients.surname, clients.address, clients.city 
FROM clients
2020-10-16 22:57:26,252 INFO sqlalchemy.engine.base.Engine {}
(1, 'Jan', 'Kowalski', 'ul. Florianska 12', 'Krakow')
(2, 'Jan', 'Kowalski', 'ul. Florianska 12', 'Krakow')
2020-10-16 22:57:26,257 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-16 22:57:26,259 INFO sqlalchemy.engine.base.Engine SELECT cars.car_id AS cars_car_id, cars.producer AS cars_producer, cars.model AS cars_model, cars.year AS cars_year, cars.horse_power AS cars_horse_power, cars.price_per_day AS cars_price_per_day 
FROM cars
2020-10-16 22:57:26,261 INFO sqlalchemy.engine.base.Engine {}
<Car: id=1, producer=Seat,model=Leon, year=2016,horse_power=80, price_per_day=200>
<Car: id=2, producer=Seat,model=Leon, year=2016,horse_power=80, price_per_day=200>

Stwórzmy teraz możliwość dodawania dużej ilośc danych poprzez funkcje insert_data

In [ ]:
def insert_data(session, base, params):
    session.add(base(**params))
    session.commit()

clients = [
    {'name': 'Jan', 'surname': 'Kowalski', 'address': 'ul. Florianska 12', 'city': 'Krakow'},
    {'name': 'Andrzej', 'surname': 'Nowak', 'address': 'ul. Saska 43', 'city': 'Wroclaw'},
    {'name': 'Michal', 'surname': 'Taki', 'address': 'os. Srodkowe 12', 'city': 'Poznan'},
    {'name': 'Pawel', 'surname': 'Ktory', 'address': 'ul. Stara 11', 'city': 'Gdynia'},
    {'name': 'Anna', 'surname': 'Inna', 'address': 'os. Srednie 1', 'city': 'Gniezno'},
    {'name': 'Alicja', 'surname': 'Panna', 'address': 'os. Duze 33', 'city': 'Torun'},
    {'name': 'Damian', 'surname': 'Papa', 'address': 'ul. Skosna 66', 'city': 'Warszawa'},
    {'name': 'Marek', 'surname': 'Troska', 'address': 'os. Male 90', 'city': 'Radom'},
    {'name': 'Jakub', 'surname': 'Klos', 'address': 'os. Polskie 19', 'city': 'Wadowice'},
    {'name': 'Lukasz', 'surname': 'Lis', 'address': 'os. Podlaskie 90', 'city': 'Bialystok'}]
cars = [
    {'producer': 'Seat', 'model': 'Leon', 'year': 2016, 'horse_power': 80, 'price_per_day': 200},
    {'producer': 'Toyota', 'model': 'Avensis', 'year': 2014, 'horse_power': 72, 'price_per_day': 100},
    {'producer': 'Mercedes', 'model': 'CLK', 'year': 2018, 'horse_power': 190, 'price_per_day': 400},
    {'producer': 'Hyundai', 'model': 'Coupe', 'year': 2014, 'horse_power': 165, 'price_per_day': 300},
    {'producer': 'Dacia', 'model': 'Logan', 'year': 2015, 'horse_power': 103, 'price_per_day': 150},
    {'producer': 'Saab', 'model': '95', 'year': 2012, 'horse_power': 140, 'price_per_day': 140},
    {'producer': 'BMW', 'model': 'E36', 'year': 2007, 'horse_power': 110, 'price_per_day': 80},
    {'producer': 'Fiat', 'model': 'Panda', 'year': 2016, 'horse_power': 77, 'price_per_day': 190},
    {'producer': 'Honda', 'model': 'Civic', 'year': 2019, 'horse_power': 130, 'price_per_day': 360},
    {'producer': 'Volvo', 'model': 'XC70', 'year': 2013, 'horse_power': 180, 'price_per_day': 280}]
bookings = [
    {'client_id': 3, 'car_id': 3, 'start_date': '2020-07-06', 'end_date': '2020-07-08', 'total_amount': 400},
    {'client_id': 6, 'car_id': 10, 'start_date': '2020-07-10', 'end_date': '2020-07-16', 'total_amount': 1680},
    {'client_id': 4, 'car_id': 5, 'start_date': '2020-07-11', 'end_date': '2020-07-14', 'total_amount': 450},
    {'client_id': 5, 'car_id': 4, 'start_date': '2020-07-11', 'end_date': '2020-07-13', 'total_amount': 600},
    {'client_id': 7, 'car_id': 3, 'start_date': '2020-07-12', 'end_date': '2020-07-14', 'total_amount': 800},
    {'client_id': 5, 'car_id': 7, 'start_date': '2020-07-14', 'end_date': '2020-07-17', 'total_amount': 240},
    {'client_id': 3, 'car_id': 8, 'start_date': '2020-07-14', 'end_date': '2020-07-16', 'total_amount': 380},
    {'client_id': 5, 'car_id': 9, 'start_date': '2020-07-15', 'end_date': '2020-07-18', 'total_amount': 1080},
    {'client_id': 6, 'car_id': 10, 'start_date': '2020-07-16', 'end_date': '2020-07-20', 'total_amount': 1120},
    {'client_id': 8, 'car_id': 1, 'start_date': '2020-07-16', 'end_date': '2020-07-19', 'total_amount': 600},
    {'client_id': 9, 'car_id': 2, 'start_date': '2020-07-16', 'end_date': '2020-07-21', 'total_amount': 500},
    {'client_id': 10, 'car_id': 6, 'start_date': '2020-07-17', 'end_date': '2020-07-19', 'total_amount': 280},
    {'client_id': 1, 'car_id': 9, 'start_date': '2020-07-17', 'end_date': '2020-07-19', 'total_amount': 720},
    {'client_id': 3, 'car_id': 7, 'start_date': '2020-07-18', 'end_date': '2020-07-21', 'total_amount': 240},
    {'client_id': 5, 'car_id': 4, 'start_date': '2020-07-18', 'end_date': '2020-07-22', 'total_amount': 1200}]

for client in clients:
    insert_data(session, Clients, client)
for car in cars:
    insert_data(session, Cars, car)
for booking in bookings:
    insert_data(session, Bookings, booking)

2020-10-16 23:00:51,022 INFO sqlalchemy.engine.base.Engine INSERT INTO clients (name, surname, address, city) VALUES (%(name)s, %(surname)s, %(address)s, %(city)s)
2020-10-16 23:00:51,025 INFO sqlalchemy.engine.base.Engine {'name': 'Jan', 'surname': 'Kowalski', 'address': 'ul. Florianska 12', 'city': 'Krakow'}
2020-10-16 23:00:51,029 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-16 23:00:51,039 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-16 23:00:51,043 INFO sqlalchemy.engine.base.Engine INSERT INTO clients (name, surname, address, city) VALUES (%(name)s, %(surname)s, %(address)s, %(city)s)
2020-10-16 23:00:51,044 INFO sqlalchemy.engine.base.Engine {'name': 'Andrzej', 'surname': 'Nowak', 'address': 'ul. Saska 43', 'city': 'Wroclaw'}
2020-10-16 23:00:51,047 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-16 23:00:51,057 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-16 23:00:51,058 INFO sqlalchemy.engine.base.Engine INSERT INTO clients (name, surname, 

Spróbujmy teraz podziałać na tej utworzonej bazie:

In [ ]:
#1
result = session.query(Bookings).filter(Bookings.client_id == 3)
for booking in result:
    print(booking)

#2
from sqlalchemy.sql import select

conn = eng.connect()
s = select([Bookings]).where(Bookings.client_id == 3)
result = conn.execute(s).fetchall()
print(result)

from sqlalchemy.sql import select
from sqlalchemy import join

j = join(Bookings, Cars, Bookings.car_id == Cars.car_id)
s = select([Cars]).select_from(j).where(Bookings.client_id == 5)
result = conn.execute(s)
for car in result:
    print(car)



2020-10-16 23:41:16,546 INFO sqlalchemy.engine.base.Engine SELECT bookings.booking_id AS bookings_booking_id, bookings.client_id AS bookings_client_id, bookings.car_id AS bookings_car_id, bookings.start_date AS bookings_start_date, bookings.end_date AS bookings_end_date, bookings.total_amount AS bookings_total_amount 
FROM bookings 
WHERE bookings.client_id = %(client_id_1)s
2020-10-16 23:41:16,547 INFO sqlalchemy.engine.base.Engine {'client_id_1': 3}
2020-10-16 23:41:16,557 INFO sqlalchemy.engine.base.Engine SELECT bookings.booking_id, bookings.client_id, bookings.car_id, bookings.start_date, bookings.end_date, bookings.total_amount 
FROM bookings 
WHERE bookings.client_id = %(client_id_1)s
2020-10-16 23:41:16,558 INFO sqlalchemy.engine.base.Engine {'client_id_1': 3}
[(1, 3, 3, datetime.date(2020, 7, 6), datetime.date(2020, 7, 8), 400), (7, 3, 8, datetime.date(2020, 7, 14), datetime.date(2020, 7, 16), 380), (14, 3, 7, datetime.date(2020, 7, 18), datetime.date(2020, 7, 21), 240)]
2020-

In [ ]:
import pickle

#Here's an example dict
grades = { 'Alice': 89, 'Bob': 72, 'Charles': 87 }

#Use dumps to convert the object to a serialized string
serial_grades = pickle.dumps( grades )
print(serial_grades)
#Use loads to de-serialize an object
received_grades = pickle.loads( serial_grades )
print(received_grades)

b'\x80\x03}q\x00(X\x05\x00\x00\x00Aliceq\x01KYX\x03\x00\x00\x00Bobq\x02KHX\x07\x00\x00\x00Charlesq\x03KWu.'
{'Alice': 89, 'Bob': 72, 'Charles': 87}


In [ ]:
import sys
!pip install -U pip
!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

from google.colab import drive
drive.mount('/drive')


Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.3)
Requirement already up-to-date: pandas-profiling[notebook] in /usr/local/lib/python3.6/dist-packages (2.9.0)
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Mounted at /drive


In [ ]:
import os
os.chdir("drive")
os.chdir("My Drive")

bin	 datalab  etc	lib32  mnt   root  srv	  tensorflow-1.15.2  usr
boot	 dev	  home	lib64  opt   run   swift  tmp		     var
content  drive	  lib	media  proc  sbin  sys	  tools
'My Drive'


In [ ]:
!ls

'1576418255304_1576416877008_skrypt- dzien-1.gdoc'
'1576418413874_skrypt- dzien-2.gdoc'
'2019.01.29_ MB workflow-Micro-solutions_Silvija.pptx'
 2019-12-13-Autenti-DR___Wroc_aw___zlecenieUZ___Damian___Brzoza___1951.gdoc
 2019-12-13-Autenti-DR___Wroc_aw___zlecenieUZ___Damian___Brzoza___1951.pdf
 20200425_114737.jpg
'7_kluczowe_zasoby (osobowe).gdoc'
'air.com.rosettastone.mobile.CoursePlayer_v5.11.2_b5110228_[SAP].apk'
'Apeer issues.gdoc'
'Arkusz kalkulacyjny bez tytułu.gsheet'
 asd.gdoc
'Barania z Wiki'
 całość.docx
 całość.pdf
 Codewars.gdoc
'Colab Notebooks'
'DamianBrzozaCV_PL_ONEPAGE (1).docx'
'DamianBrzozaCV_PL_ONEPAGE (1).gdoc'
 DamianBrzozaCV_PL_ONEPAGE.docx
 DamianBrzozaCV_PL_ONEPAGE.gdoc
'Dane Dane Dane.gdoc'
 Database.kdbx
 DataScienceInterviewQ.pdf
'data science jola.pptx'
 Data+Science-kopia.pptx
'Data Science zdalny RD-skonwertowany.pptx'
'Diagnostyka to Gówno.gdoc'
'Dla Pauliny KNIAŚ'
 Documents
'Dokument bez nazwy.gdoc'
'Dokument bez tsadsadsaytułu.gdoc'
'Dokument bez

In [ ]:
# Standard Library Imports
from pathlib import Path

# Installed packages
import pandas as pd
from ipywidgets import widgets

# Our package
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

In [ ]:
# Read the Titanic Dataset
file_name = cache_file(
    "titanic.csv",
    "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv",
)
df = pd.read_csv(file_name)

In [ ]:
# Generate the Profiling Report
profile = ProfileReport(df, title="Titanic Dataset", html={'style': {'full_width': True}}, sort="None")

In [ ]:
# The Notebook Widgets Interface
profile.to_file('report.html')